In [1]:
import pandas as pd
import numpy as np
import json
import ast
import os
import re

In [2]:
def format_lyrics(lyrics):
    lyrics = re.sub('[.,!:?\"\'«»]', '', lyrics)
    lyrics = re.sub('[-–—— ]+', ' ', lyrics)
    lyrics = lyrics.strip()
    lyrics = lyrics.lower()
    return lyrics

In [3]:
songs = pd.read_csv('songs_dataset.csv', index_col=0)
songs.head()

,Author,Album,Song,Featuring,Lyrics
0,$uicideBoy$,I Want to Die in New Orleans,King Tulip,[],"\n\n[Interlude: Max Beck]\n""They changed so mu..."
1,$uicideBoy$,I Want to Die in New Orleans,Bring Out Your Dead,[],\n\n[Intro: Koopsta Knicca & YUNG MUTT]\nYou g...
2,$uicideBoy$,I Want to Die in New Orleans,Nicotine Patches,[],\n\n[Intro (Distorted)]\nTriple 6 'till death\...
3,$uicideBoy$,I Want to Die in New Orleans,"10,000 Degrees",[],"\n\n[Intro: Lil Gin]\nLoad up my big nine, now..."
4,$uicideBoy$,I Want to Die in New Orleans,122 Days,[],"\n\n[Verse 1: LORD OF LONELINESS]\nUh, local b..."


In [4]:
songs.shape

(34753, 5)

In [5]:
def get_part_lyrics(part):
    part = part[1:]
    return '\n'.join(part)

In [6]:
def get_part_info(part, all_singers, author, album, song):
    singer = None
    brackets = part[0]
    brackets_parts = brackets[1:-1].split(':')
    if len(brackets_parts) == 1:
        singer = all_singers[0]
        part_name = brackets_parts[0]
    else:
        part_singers = brackets_parts[1]
        part_singers = list(map(str.strip, re.split(',|&', part_singers)))
        for part_singer in part_singers:
            if part_singer in all_singers:
                singer = part_singer
                break
        part_name = brackets_parts[0]
    part_lyrics = get_part_lyrics(part)
    if not singer:
        singer = author
    if part_lyrics == '':
        return None
    return [author, album, song, singer, part_name, part_lyrics]

In [7]:
def get_parts_from_song(song):
    feats = ast.literal_eval(song['Featuring'])
    author, album, song, _, lyrics = song.values
    all_singers = [author] + feats
    print(author, '-', album, '-', song, '-', feats, '-', all_singers)
    
    indices = [i for i, ltr in enumerate(lyrics) if ltr == '['] + [len(lyrics)]
    song_parts = [lyrics[indices[i]:indices[i+1]].strip().split('\n') for i, index in enumerate(indices[:-1])]
    song_parts_info = [get_part_info(song_part, all_singers, author, album, song) for song_part in song_parts]
    song_parts_info = list(filter(None, song_parts_info))

    df = pd.DataFrame(song_parts_info, columns=['Author', 'Album', 'Song', 'Part Singer', 'Part Name', 'Part Lyrics'])
    return df

In [8]:
%%time
parts = songs.apply(get_parts_from_song, axis=1)
parts = pd.concat(parts.tolist(), axis=0)

$uicideBoy$ - I Want to Die in New Orleans - King Tulip - [] - ['$uicideBoy$']
$uicideBoy$ - I Want to Die in New Orleans - King Tulip - [] - ['$uicideBoy$']
$uicideBoy$ - I Want to Die in New Orleans - Bring Out Your Dead - [] - ['$uicideBoy$']
$uicideBoy$ - I Want to Die in New Orleans - Nicotine Patches - [] - ['$uicideBoy$']
$uicideBoy$ - I Want to Die in New Orleans - 10,000 Degrees - [] - ['$uicideBoy$']
$uicideBoy$ - I Want to Die in New Orleans - 122 Days - [] - ['$uicideBoy$']
$uicideBoy$ - I Want to Die in New Orleans - Phantom Menace - [] - ['$uicideBoy$']
$uicideBoy$ - I Want to Die in New Orleans - Krewe du Vieux (Comedy & Tragedy) - [] - ['$uicideBoy$']
$uicideBoy$ - I Want to Die in New Orleans - WAR TIME ALL THE TIME - [] - ['$uicideBoy$']
$uicideBoy$ - I Want to Die in New Orleans - Coma - [] - ['$uicideBoy$']
$uicideBoy$ - I Want to Die in New Orleans - Long Gone (Save Me from This Hell) - [] - ['$uicideBoy$']
$uicideBoy$ - I Want to Die in New Orleans - Meet Mr. NICE

$uicideBoy$ - Gray/Grey - To Kill a Mockingbird - ['Nightmare Nasty'] - ['$uicideBoy$', 'Nightmare Nasty']
$uicideBoy$ - Gray/Grey - Clouds As Witnesses - [] - ['$uicideBoy$']
$uicideBoy$ - Gray/Grey - Gaudy Pack Shawty - ['Yung God'] - ['$uicideBoy$', 'Yung God']
$uicideBoy$ - Gray/Grey - Behold a Pale Horse - [] - ['$uicideBoy$']
$uicideBoy$ - Gray/Grey - Kill Yourself (Part II) - [] - ['$uicideBoy$']
$uicideBoy$ - Gray/Grey - Gray/Grey (Bonus Album Only Track) - [] - ['$uicideBoy$']
$uicideBoy$ - KILL YOURSELF Part X: The Resurrection Saga - Durango ’95 (A Real Horror Show) - [] - ['$uicideBoy$']
$uicideBoy$ - KILL YOURSELF Part X: The Resurrection Saga - Lemon Slime - [] - ['$uicideBoy$']
$uicideBoy$ - KILL YOURSELF Part X: The Resurrection Saga - Smoked Out, Loced Out (Part II) - ['Eddy Baker'] - ['$uicideBoy$', 'Eddy Baker']
$uicideBoy$ - KILL YOURSELF Part X: The Resurrection Saga - Flodgin’ - [] - ['$uicideBoy$']
$uicideBoy$ - KILL YOURSELF Part X: The Resurrection Saga - Limp 

2 Chainz - #MEALTIME - Chicken Marsala - [] - ['2 Chainz']
2 Chainz - #MEALTIME - Mixed Seafood Grill Kebabs - [] - ['2 Chainz']
2 Chainz - #MEALTIME - Shrimp & Grits - [] - ['2 Chainz']
2 Chainz - #MEALTIME - Sautéed Asparagus / “Me Time” Sauce - [] - ['2 Chainz']
2 Chainz - #MEALTIME - Garlicky Green Beans / Smoky Kale Greens - [] - ['2 Chainz']
2 Chainz - #MEALTIME - Pan-Seared Sea Bass With Heirloom Tomato Salsa - [] - ['2 Chainz']
2 Chainz - #MEALTIME - Shrimp Scampi - [] - ['2 Chainz']
2 Chainz - B.O.A.T.S. II #METIME - Fork - [] - ['2 Chainz']
2 Chainz - B.O.A.T.S. II #METIME - 36 - [] - ['2 Chainz']
2 Chainz - B.O.A.T.S. II #METIME - Feds Watching - ['Pharrell Williams'] - ['2 Chainz', 'Pharrell Williams']
2 Chainz - B.O.A.T.S. II #METIME - Where U Been - ['Cap-1'] - ['2 Chainz', 'Cap-1']
2 Chainz - B.O.A.T.S. II #METIME - I Do It - ['Drake', 'Lil Wayne'] - ['2 Chainz', 'Drake', 'Lil Wayne']
2 Chainz - B.O.A.T.S. II #METIME - Used 2 - [] - ['2 Chainz']
2 Chainz - B.O.A.T.S. II 

21 Savage - Free Guwop - H2O - [] - ['21 Savage']
21 Savage - Free Guwop - One Foot - [] - ['21 Savage']
21 Savage - Free Guwop - Supply - [] - ['21 Savage']
21 Savage - Free Guwop - Red Opps - [] - ['21 Savage']
21 Savage - Free Guwop - 12 AM - [] - ['21 Savage']
21 Savage - Free Guwop - Twenty1 - [] - ['21 Savage']
21 Savage - The Slaughter Tape - Skrrt Skrrt - [] - ['21 Savage']
21 Savage - The Slaughter Tape - Pimp - [] - ['21 Savage']
21 Savage - The Slaughter Tape - Gang - [] - ['21 Savage']
21 Savage - The Slaughter Tape - Picky - [] - ['21 Savage']
21 Savage - The Slaughter Tape - Slaughter Ya Daughter - ['ILOVEMAKONNEN', 'KEY!'] - ['21 Savage', 'ILOVEMAKONNEN', 'KEY!']
21 Savage - The Slaughter Tape - Drip - [] - ['21 Savage']
21 Savage - The Slaughter Tape - Out the Bowl - ['KEY!'] - ['21 Savage', 'KEY!']
21 Savage - The Slaughter Tape - Drinkin And Driving - [] - ['21 Savage']
21 Savage - The Slaughter Tape - Start Dying - ['Offset'] - ['21 Savage', 'Offset']
21 Savage - The

2Pac - 2Pac Live - Hit Em Up (Live) - [] - ['2Pac']
2Pac - 2Pac Live - Tattoo Tears (Live) - [] - ['2Pac']
2Pac - 2Pac Live - All About U (Live) - [] - ['2Pac']
2Pac - 2Pac Live - Never Call You Bitch Again (Live) - [] - ['2Pac']
2Pac - 2Pac Live - How Do You Want It (Live) - [] - ['2Pac']
2Pac - 2Pac Live - 2 Of Amerikaz Most Wanted (Live) - [] - ['2Pac']
2Pac - 2Pac Live - California Love (Live) - [] - ['2Pac']
2Pac - Tupac: Resurrection - Ghost - [] - ['2Pac']
2Pac - Tupac: Resurrection - One Day at a Time - ['Outlawz', 'Eminem'] - ['2Pac', 'Outlawz', 'Eminem']
2Pac - Tupac: Resurrection - Death Around the Corner - [] - ['2Pac']
2Pac - Tupac: Resurrection - Secretz of War - [] - ['2Pac']
2Pac - Tupac: Resurrection - Runnin’ (Dying to Live) - ['The Notorious B.I.G.'] - ['2Pac', 'The Notorious B.I.G.']
2Pac - Tupac: Resurrection - Holler If Ya Hear Me - [] - ['2Pac']
2Pac - Tupac: Resurrection - Starin’ Through My Rear View - [] - ['2Pac']
2Pac - Tupac: Resurrection - Bury Me a G - ['

...

The White Stripes - Elephant - Girl, You Have No Faith in Medicine - [] - ['The White Stripes']
The White Stripes - Elephant - Well It’s True That We Love One Another - ['Holly Golightly'] - ['The White Stripes', 'Holly Golightly']
The White Stripes - Elephant - Who’s to Say - [] - ['The White Stripes']
The White Stripes - Elephant - Good to Me - [] - ['The White Stripes']
The White Stripes - White Blood Cells - Dead Leaves and the Dirty Ground - [] - ['The White Stripes']
The White Stripes - White Blood Cells - Hotel Yorba - [] - ['The White Stripes']
The White Stripes - White Blood Cells - I’m Finding It Harder to Be a Gentleman - [] - ['The White Stripes']
The White Stripes - White Blood Cells - Expecting - [] - ['The White Stripes']
The White Stripes - White Blood Cells - Offend in Every Way - [] - ['The White Stripes']
The White Stripes - White Blood Cells - I Think I Smell a Rat - [] - ['The White Stripes']
The White Stripes - White Blood Cells - I Can’t Wait - [] - ['The White S

Vanilla Ice - Hard to Swallow - Freestyle - ['C-Note', '2Hype', 'Insane Poetry'] - ['Vanilla Ice', 'C-Note', '2Hype', 'Insane Poetry']
Vanilla Ice - Mind Blowin - Fame - [] - ['Vanilla Ice']
Vanilla Ice - Mind Blowin - The Wrath - [] - ['Vanilla Ice']
Vanilla Ice - Mind Blowin - Roll ’Em Up - ['The VIP Posse'] - ['Vanilla Ice', 'The VIP Posse']
Vanilla Ice - Mind Blowin - Hit ’Em Hard - [] - ['Vanilla Ice']
Vanilla Ice - Mind Blowin - Now and Forever - [] - ['Vanilla Ice']
Vanilla Ice - Mind Blowin - Iceman Party - [] - ['Vanilla Ice']
Vanilla Ice - Mind Blowin - Oh My Gosh - [] - ['Vanilla Ice']
Vanilla Ice - Mind Blowin - Minutes of Power - [] - ['Vanilla Ice']
Vanilla Ice - Mind Blowin - I Go Down - [] - ['Vanilla Ice']
Vanilla Ice - Mind Blowin - Phunky Rhymes - [] - ['Vanilla Ice']
Vanilla Ice - Mind Blowin - Blowin My Mind - [] - ['Vanilla Ice']
Vanilla Ice - Mind Blowin - Get Loose - [] - ['Vanilla Ice']
Vanilla Ice - Extremely Live - Ice Is Workin It - [] - ['Vanilla Ice']
Vani

Whitney Houston - The Star Spangled Banner / America the Beautiful - Single - The Star Spangled Banner - ['The Florida Orchestra'] - ['Whitney Houston', 'The Florida Orchestra']
Whitney Houston - The Star Spangled Banner / America the Beautiful - Single - America The Beautiful - [] - ['Whitney Houston']
Whitney Houston - I’m Your Baby Tonight - My Name Is Not Susan - [] - ['Whitney Houston']
Whitney Houston - I’m Your Baby Tonight - Lover For Life - [] - ['Whitney Houston']
Whitney Houston - I’m Your Baby Tonight - Anymore - [] - ['Whitney Houston']
Whitney Houston - I’m Your Baby Tonight - I Belong To You - [] - ['Whitney Houston']
Whitney Houston - I’m Your Baby Tonight - Who Do You Love - [] - ['Whitney Houston']
Whitney Houston - I’m Your Baby Tonight - We Didn’t Know - ['Stevie Wonder'] - ['Whitney Houston', 'Stevie Wonder']
Whitney Houston - I’m Your Baby Tonight - After We Make Love - [] - ['Whitney Houston']
Whitney Houston - I’m Your Baby Tonight - I’m Knockin - [] - ['Whitney

Wu-Tang Clan - 8 Diagrams - Wolves - ['George Clinton'] - ['Wu-Tang Clan', 'George Clinton']
Wu-Tang Clan - 8 Diagrams - Gun Will Go - ['Sunny Valentine'] - ['Wu-Tang Clan', 'Sunny Valentine']
Wu-Tang Clan - 8 Diagrams - Sunlight - [] - ['Wu-Tang Clan']
Wu-Tang Clan - 8 Diagrams - Stick Me for My Riches - ['Gerald Alston'] - ['Wu-Tang Clan', 'Gerald Alston']
Wu-Tang Clan - 8 Diagrams - Starter - ['Tash Mahogany', 'Sunny Valentine', 'Streetlife'] - ['Wu-Tang Clan', 'Tash Mahogany', 'Sunny Valentine', 'Streetlife']
Wu-Tang Clan - 8 Diagrams - Windmill - ['Cappadonna'] - ['Wu-Tang Clan', 'Cappadonna']
Wu-Tang Clan - 8 Diagrams - Weak Spot - [] - ['Wu-Tang Clan']
Wu-Tang Clan - 8 Diagrams - Life Changes - [] - ['Wu-Tang Clan']
Wu-Tang Clan - 8 Diagrams - Tar Pit - ['George Clinton', 'Cappadonna', 'Streetlife'] - ['Wu-Tang Clan', 'George Clinton', 'Cappadonna', 'Streetlife']
Wu-Tang Clan - 8 Diagrams - 16th Chamber - [] - ['Wu-Tang Clan']
Wu-Tang Clan - Think Differently Music: Wu-Tang Meet

Wall time: 50.3 s


In [9]:
parts.head()

,Author,Album,Song,Part Singer,Part Name,Part Lyrics
0,$uicideBoy$,I Want to Die in New Orleans,King Tulip,$uicideBoy$,Interlude,"""They changed so much, you know what I'm sayin..."
1,$uicideBoy$,I Want to Die in New Orleans,King Tulip,$uicideBoy$,Verse 1,"Yeah, I was the wave, but now I feel the tide ..."
2,$uicideBoy$,I Want to Die in New Orleans,King Tulip,$uicideBoy$,Verse 2,Two cups and a bad bitch\nThen I pop me a Xana...
3,$uicideBoy$,I Want to Die in New Orleans,King Tulip,$uicideBoy$,Refrain,"4 AM, praying, ""can I get some rest?""\nDreams ..."
4,$uicideBoy$,I Want to Die in New Orleans,King Tulip,$uicideBoy$,Outro,"""Happening now, streets blocked off as New Orl..."


In [10]:
parts.shape

(204690, 6)

In [11]:
parts.to_csv('parts_dataset.csv')